In [ ]:
"""
Module docstring
"""
import subprocess
import itertools
import zipfile
import os.path
import shutil
import numpy as np

import os
from pym import read_mym, write_mym
from scipy.interpolate import interp1d
from core.core import pairwise

# TODO: move version information elsewhere
# Define version information
__version__ = "0.1.1"


def create_io_settings(structure, output=""):
    """
    """
    if not output:
        output = structure.baseline_folder

    scenario_library = os.path.relpath(structure.timer_scenlib, start=structure.timer_folder)
    settings = []
    # > read in general data
    settings.append("sce timer_input.sce")
    settings.append("rsc {}".format(structure.baseline_folder))
    # > read in costcurve and baseline specific scenario files
    settings.append("sce {}".format(os.path.join(scenario_library,"scenario_input.sce")))
    settings.append("rsc {}".format(structure.baseline_folder))
    # > run scenario and generate output
    settings.append("run")
    settings.append("sce timer_costcurve_output.sce")
    settings.append("wsc {}".format(output))

    return settings

# TODO: remove, check if there are no more references
# def create_empty_folder(folder):
#     """
#     """
#     if os.path.isdir(folder):
#         shutil.rmtree(folder)
#         print("Removed existing scenario folder [{}]".format(os.path.realpath(folder)))
    
#     # On network drives it's sometimes necessary to try a few times to create a
#     # folder that has been removed recently.
#     tries = 0
#     max_tries = 10
#     while tries < max_tries:
#         tries += 1
#         try:
#             os.makedirs(os.path.abspath(folder))
#         except Exception as error:
#             continue
#         else:
#             print("Created scenario folder [{}]".format(os.path.realpath(folder)))
#             break
#     else:
#         raise(error)


def construct_scenario(baseline, structure):
    """
    Construct scenario from all relevant scenario files. Return list.
    """
    # TODO: remove line below
    # create_empty_folder(structure.scenario_zip_folder)

    temp_folder = os.path.join(structure.scenario_zip_folder, baseline.name)
    os.makedirs(temp_folder, exist_ok=True)
    
    # TODO: check if and how to include the following in either `baseline` or
    # `structure`
    baseline_path = os.path.join(structure.root_folder, "scenlib", structure.timer_version, "baselines")
    baseline_ordered_blocks = dict.fromkeys(
        [structure.baseline.kernel, structure.baseline.root, structure.baseline.name]
        )
    baseline_ordered_folders = [os.path.join(baseline_path, block)
                                for block in baseline_ordered_blocks.keys()]
    # baseline_folders.append(structure.timer_scenlib)

    # TODO: now plainly copying all scenario files. Probably not correct. Check
    # necessary changes here
    for baseline_folder in baseline_ordered_folders:
        for root, folders, files in os.walk(baseline_folder):
            print(root)
            if ".svn" in folders:
                folders.remove(".svn")
            for folder in folders:
                source_folder = os.path.join(root, folder)
                relative_folder = os.path.relpath(source_folder, baseline_folder)
                target_folder = os.path.join(temp_folder, relative_folder)
                os.makedirs(target_folder, exist_ok=True)
                shutil.copystat(source_folder, target_folder)
            for file in files:
                source_file = os.path.join(root, file)
                relative_file = os.path.relpath(source_file, baseline_folder)
                target_file = os.path.join(temp_folder, relative_file)
                shutil.copy2(source_file, target_file)
    
    settings_files = [os.path.join(folder, "scenario_settings.bat") for folder in baseline_ordered_folders]

    comments, settings = [], []
    echo = "echo"
    rem = "rem"
    write_to = ">"
    for filename in settings_files:
        try:
            with open(filename) as settings_file:
                raw_settings = settings_file.readlines()
        except OSError:
            print("Could not read [{}]".format(filename))
            continue

        # Process lines in `settings_files`
        for line in raw_settings:
            line = line.strip().lower()
            if line:
                if line.startswith(rem):
                    comments.append(line)
                elif line.startswith(echo):
                    # strip parts that write to in-file
                    line = line[len(echo):line.find(write_to)].strip()
                    settings.append(line)
                else:
                    print("Uh oh: ", line[:10])
    
    # TODO: check if we need to write this to file
    settings_filename = os.path.join(structure.scenario_zip_folder, "settings.scn")
    with open(settings_filename, mode="w") as settings_file:
        settings_file.write("\n".join(settings))
    
    scenario_files = [os.path.join(folder, "scenario_input.sce") for folder in baseline_ordered_folders]
    scenario_input = []
    for filename in scenario_files:
        try:
            with open(filename) as scenario_file:
                scenario_input.extend(scenario_file.readlines())
        except OSError:
            print("Could not read [{}]".format(filename))
            continue

    scenario_filename = os.path.join(temp_folder, "scenario_input.sce")
    with open(scenario_filename, mode="w") as scenario_file:
        scenario_file.write("".join(scenario_input))

    return settings


def create_output_scenario_file(structure):
    """
    """
    # TODO use version comparison from distutils based on version naming?
    # TODO: use logging
    # TODO: move this version comparison to structure?
    # TODO: save directly to timer folder?
    if structure.timer_version.lower() in ["timer_2015", "timer_2013", "timer_2011"]:
        co2_variable_name = "CO2EST"
    elif structure.timer_version.lower() in ["timer_3_11", "timer_32"]:
        co2_variable_name = "CO2ESTCC"
    else:
        co2_variable_name = "CO2ESTCC"
        print("warning: TIMER version not recognized, using settings for timer_3_11")

    lines = [
    '! Output for generating costcurves',
    'DIRECTORY("../outputlib/$1");',
    'FILE("co2tax","w") = main.em.{};'.format(co2_variable_name),
    'FILE("cementco2tax","w") = main.em.mis.ECO2CEMENT_forFAIR;',
    'FILE("n2otax","w") = main.em.mis.enemn2o;',
    'FILE("tax","w") = main.em.carbontax;',
    'FILE("sf6tax","w") = main.em.mis.HM.HC.enemsf6;',
    'FILE("pfccf4tax","w") = main.em.mis.HM.HC.enempfccf4;',
    'FILE("pfcc2f6tax","w") = main.em.mis.HM.HC.enempfcc2f6;',
    'FILE("pfcc6f14tax","w") = main.em.mis.HM.HC.enempfcc6f14;',
    'FILE("hfc4310tax","w") = main.em.mis.HM.HC.enemhfc4310;',
    'FILE("hfc245tax","w") = main.em.mis.HM.HC.enemhfc245;',
    'FILE("hfc236tax","w") = main.em.mis.HM.HC.enemhfc236;',
    'FILE("hfc227tax","w") = main.em.mis.HM.HC.enemhfc2NRC;',
    'FILE("hfc152tax","w") = main.em.mis.HM.HC.enemhfc152;',
    'FILE("hfc143tax","w") = main.em.mis.HM.HC.enemhfc143;',
    'FILE("hfc134tax","w") = main.em.mis.HM.HC.enemhfc134;',
    'FILE("hfc125tax","w") = main.em.mis.HM.HC.enemhfc125;',
    'FILE("hfc32tax","w") = main.em.mis.HM.HC.enemhfc32;',
    'FILE("hfc23tax","w") = main.em.mis.HM.HC.enemhfc23;']
    
    output_scenario_filepath = os.path.join(
        structure.script_folder, "..", "task_files", "timer_costcurve_output.sce")

    with open(output_scenario_filepath, "w") as output_scenario_file:
        output_scenario_file.write("\n".join(lines))


def zip_model(zip_filename, structure, compile_matlab=False):
    """
    Zip all and only relevant model files for a TIMER run.
    """
    # Check whether output file and folder exist, act accordingly
    # zip_output_folder = os.path.normpath(os.path.join(output_folder, "model"))
    zip_filepath = os.path.join(structure.timer_zip_folder, zip_filename)
    # model_root_folder = os.path.join(model_folder, "..")
    
    # TODO: perhaps create a generate_folder_structure method to create a
    # costcurve folder structure -> available in `CostcurveStructure`, see if we
    # need to build in check to see  if folder really is empty
    # os.makedirs(os.path.abspath(structure.timer_zip_folder), exist_ok=True)
    # if os.path.isfile(zip_filepath):
    #     os.remove(zip_filepath)
    #     print("Removed existing {}".format(zip_filename))
    
    # Write specific set of folders to zip file with [zip_name]
    with zipfile.ZipFile(zip_filepath ,"w") as model_zipfile:
        # > zip the model folders, with its subfolders
        for root, folders, files in os.walk(structure.timer_folder):
            for filename in files:
                filepath = os.path.join(root, filename)
                relative_filepath = os.path.relpath(filepath, start=structure.timer_folder)
                archive_name = os.path.join("timer", relative_filepath)
                model_zipfile.write(filepath, arcname=archive_name)
            # > do not zip unnecessary subversion data
            if ".svn" in folders:
                folders.remove(".svn")

    print("Files zipped to {}.".format(zip_filename))
    print()


def zip_scenario(zip_filename, structure):
    """
    """
    zip_filepath = os.path.join(structure.scenario_zip_folder, zip_filename)

    # TODO: improve this - bad coding. temp_folder also defined in construct_scenario
    temp_folder = os.path.join(structure.scenario_zip_folder, structure.baseline.name)

    # Write specific set of folders to zip file with [zip_name]
    with zipfile.ZipFile(zip_filepath ,"w") as scenario_zipfile:
        # > zip the model folders, with its subfolders
        for root, folders, files in os.walk(temp_folder):
            for filename in files:
                filepath = os.path.join(root, filename)
                relative_filepath = os.path.relpath(filepath, start=temp_folder)
                archive_name = os.path.join("scenlib", structure.baseline_folder, relative_filepath)
                scenario_zipfile.write(filepath, arcname=archive_name)
            # > do not zip unnecessary subversion data
            if ".svn" in folders:
                folders.remove(".svn")

    # TODO: improve this, now remove by default
    # remove `temp_folder` to avoid it being copied to the cloud storage
    shutil.rmtree(temp_folder)


def compile_timer(source_path):
    """
    Compile TIMER model.
    """
    compiler = os.path.join(source_path, "compile_cc.bat")
    
    print("Compiling TIMER")
    
    compiled = subprocess.run([compiler],
        cwd=source_path, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if not compiled.stderr.decode():
        print("Succesfully compiled TIMER")
    else:
        print("An error occurred while compiling TIMER:")
        print(compiled.stderr.decode())


def live_compile(command, source_path):
    print("View here your live results!")
    command= os.path.join(source_path, command)
    process = subprocess.Popen(command, cwd=source_path, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=False)
    while True:
        line = process.stdout.readline().decode().rstrip()
        print(line)
        if not process.poll() == None:
            break

    # with open('test.log', 'w') as f:  # replace 'w' with 'wb' for Python 3
    #     process = subprocess.Popen(your_command, stdout=subprocess.PIPE)
    #     for line in iter(process.stdout.readline, ''):  # replace '' with b'' for Python 3
    #         sys.stdout.write(line)
    #         f.write(line)


def get_revision(working_copy):
    """
    Gets revision number of working copy. Returns int.
    """
    revision = subprocess.run("svn info --show-item last-changed-revision",
        cwd=working_copy, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if not revision.stderr.decode():
        return int(revision.stdout.decode().strip())
    else:
        print("An error occurred while compiling TIMER:")
        print(revision.stderr.decode())
        return -1


def prepare_costcurve(baseline, structure):
    """
    Prepare scenario data files and settings to run costcurves.
    """
    # Prepare TIMER
    # TODO: process revision number(s), currently only `timer_revision` available,
    # but unprocessed

    # TODO: combine the code in `create_output_scenario_file` with this compiler code below, elsewhere.
    # TODO use version comparison from distutils based on version naming?
    # TODO: use logging
    # TODO: move this version comparison to structure?
    if structure.timer_version.lower() in ["timer_2015", "timer_2013", "timer_2011"]:
        compiler = "compile_noelec.bat"
    elif structure.timer_version.lower() in ["timer_3_11", "timer_32"]:
        compiler = "compile.bat"
    else:
        compiler = "compile.bat"
        print("warning: TIMER version not recognized, using settings for timer_3_11")

    timer_revision = get_revision(structure.timer_folder)
    timer_zip_name = "{}.zip".format(structure.timer_version)
    live_compile(compiler, structure.timer_folder)
    create_output_scenario_file(structure)
    zip_model(timer_zip_name, structure)

    # Prepare costcurve baseline scenario
    settings = construct_scenario(baseline=baseline, structure=structure)
    scenario_zip_name = "scenario_{}.zip".format(structure.baseline.name) 
    zip_scenario(scenario_zip_name, structure)

    return settings


def calculate_costcurves(structure, tax):
    """
    Calculate costcurves from output of individual TIMER runs.

    Parameters
    ----------
    structure : CostcurveStructure
        project environment folder structure
    tax : CarbonTax
        carbon tax pathways specification

    Returns
    -------
    costcurve : dict
    """
    # Supply the names of `all_gases`, `hfcgases` and `pfcgases` are specified
    # separately as they are stored as a single variable
    gases = ["co2", "cementco2", "sf6"]
    hfcgases = ["hfc23", "hfc32", "hfc4310", "hfc125", "hfc134", "hfc143", 
                "hfc152", "hfc227", "hfc236", "hfc245"]
    pfcgases= ["pfccf4", "pfcc2f6", "pfcc6f14"]

    all_gases = gases + hfcgases + pfcgases
    variables = gases + ["hfc", "pfc"]

    costcurve = {}
    costcurve_abs = {}
    epsilon = 0.001

    metadata = "costcurve generated with costcurve cooker version {}".format(__version__)

    # moving average period
    period = 20
    # TODO: make sure relative and absolute are only calculated when required
    for curve_type in tax.curves:

        source_folder = os.path.join(structure.output_folder, curve_type)
        

        for gas in all_gases:

            raw_curve = np.zeros(tax.raw_curve_shape)

            for year, level in itertools.product(tax.years, tax.rawlevels):
                # read file and assign data of final `year` to `raw_curve`
                filename = "{}tax({})_({}).dat".format(gas, year, level)
                emissions, time = read_mym(filename, path=source_folder)
                raw_curve[tax.years == year, tax.rawlevels == level, :] = emissions[time == year, :]
            
            # calculate relative difference compared to zero-ctax baseline
            # TODO: change out parameter, as when baseline == 0 -> curve1 == 0
            baseline = raw_curve[:, tax.rawlevels == 0, :]
            relative_curve = 100 * (1 - np.divide(raw_curve, baseline, where=(baseline != 0),
                                                  out=np.ones(raw_curve.shape)))
            
            # interpolate `relative_curve` to get a costcurve with a regular tax interval
            f = interp1d(tax.rawlevels, relative_curve, axis=1, kind="linear")
            relative_curve = f(tax.targetlevels)
            
            # calculate absolute difference compared to zero-ctax baseline
            curve_abs = np.subtract(baseline, raw_curve, out=np.zeros(raw_curve.shape))
            
            # interpolate `curve_abs` to get a costcurve with a regular tax interval
            f_abs = interp1d(tax.rawlevels, curve_abs, axis=1, kind="linear")
            curve_abs = f_abs(tax.targetlevels)
            
            # make curve array
            curve = np.zeros(tax.final_curve_shape)

            # a marginal abatement costcurve must (marginally) increase with
            # increasing tax levels. Start at `level_idx == 1`, as the first
            # level does not need to be corrected
            indices = itertools.product(range(tax.nyears), range(1, tax.ntargetlevels), range(tax.nregions))
            for index in indices:
                year_idx, level_idx, region_idx = [*index]
                 
                # moving average 
                start = max(1, level_idx - period//2)
                stop  = min(tax.ntargetlevels, level_idx + period//2)
                window = slice(start, stop)
                curve[index] = np.average(relative_curve[year_idx, window, region_idx])
                
                # correction making sure no decline
                if 0 < curve[index] <= curve[year_idx, level_idx-1, region_idx]:
                    curve[index] = curve[year_idx, level_idx-1, region_idx] + epsilon

            costcurve[(gas, curve_type)] = curve
            costcurve_abs[(gas, curve_type)] = curve_abs

        hfc_curves = [costcurve_abs[(gas, curve_type)] for gas in hfcgases]
        costcurve[("hfc", curve_type)] = np.stack(hfc_curves, axis=1)

        pfc_curves = [costcurve_abs[(gas, curve_type)] for gas in pfcgases]
        costcurve[("pfc", curve_type)] = np.stack(pfc_curves, axis=1)

        for variable in variables:
            # f-gases only have linear costcurves
            if (curve_type in ["cubic", "cubicroot"]) and (variable in ["hfc", "pfc", "sf6"]):
                continue
            variable_name = "costcurve_{}_{}".format(variable, curve_type)        
            write_mym(
                costcurve[(variable, curve_type)], years=tax.years, table="wide", 
                variable_name=variable_name, folder=structure.output_foldersvn,
                comment=metadata)
        
    return costcurve

if __name__ == "__main__":

    timer_branch = "TIMER_2015_SSPs"
    timer_version = "TIMER_2015"
    project = "SSPs"
    baseline_kernel = "SSP2"
    baseline_root = "SSP2"
    baseline_energy = "LIRE"
    baseline_land = "SSP1_REDD10reforFullP"

    from main_costcurve import Baseline, CostcurveStructure, CarbonTax

    baseline = Baseline(
        baseline_kernel=baseline_kernel,
        baseline_root=baseline_root,
        baseline_energy=baseline_energy,
        baseline_land=baseline_land
        )
    
    structure = CostcurveStructure(
        project=project,
        timer_version=timer_version,
        timer_branch=timer_branch,
        baseline=baseline
        )
    
    tax_intervals = [0, 400, 2000, 4000]
    tax_samples = [20, 40, 100]
    time_interval = [2020, 2100]
    time_sample = 10
    ntargetlevels = 201

    tax = CarbonTax(time_interval, time_sample, tax_intervals, tax_samples, ntargetlevels)
    # construct_scenario(baseline=baseline, structure=structure)
    # scenario_zip_name = "scenario_{}".format(structure.baseline.name) 
    # zip_scenario(zip_filename=scenario_zip_name, structure=structure)
    calculate_costcurves(structure=structure, tax=tax)

    # :RENAMINGMODELZIP
    # echo off
    # echo.
    # echo Renaming model-zip with SVN-revision
    # echo Renaming model-zip with SVN-revision >> %log% 2>&1
    # sleep 3
    # set local
    # cd /d %timer%\%TIMERVersion%
    # svn info |grep "Last Changed Rev:" |cut -c19- >> "temp"
    # set /p svnrev=<"temp"
    # del "temp"
    # cd /d %timer%
    # ren %TIMERVersion%.zip %TIMERVersion%_r%svnrev%.zip
    # set local 

    # :CHECKIFRPROJECTFOLDEREXIST
    # if exist R:\model\TIMER_cc\user\%grid_user%\%project% goto COPYMODELZIPTOR
    # mkdir %timerccdir%
    # mkdir %timerccdir%\scenlib
    # mkdir %timerccdir%\zips

    # :COPYMODELZIPTOR
    # echo.
    # echo Copy model-zip to R location
    # echo Copy model-zip to R location >> %log% 2>&1
    # sleep 3
    # setlocal
    # cd /d %timer%
    # copy %TIMERVersion%_r%svnrev%.zip %timerccdir%\zips\%TIMERVersion%_r%svnrev%.zip
    # setlocal

    # :COPYTRUNKSCENTOR
    # echo off
    # echo.
    # echo Copy trunk to R location: %trunk%
    # echo Copy trunk to R location: %trunk% >> %log% 2>&1
    # sleep 3
    # if exist %timerccdir%\scenlib\%fscbase% rmdir %timerccdir%\scenlib\%fscbase% /s /q
    # xcopy /y %timer%\scenlib\%TIMERGeneration%\baselines\%trunk% %timerccdir%\scenlib\%fscbase% /s /e /i

    # :COPYBASESCENTOR
    # echo.
    # echo Copy second baseline to R location: %base_scen%
    # echo Copy second baseline to R location: %base_scen% >> %log% 2>&1
    # sleep 3
    # xcopy /y %timer%\scenlib\%TIMERGeneration%\baselines\%base_scen% %timerccdir%\scenlib\%fscbase% /s /e /i

    # :COPYBIOBASESCENTOR
    # echo.
    # echo Copy Bio baseline to R location: %bio_base%
    # echo Copy Bio baseline to R location: %bio_base% >> %log% 2>&1
    # sleep 3
    # xcopy /y %timer%\scenlib\%TIMERGeneration%\baselines\%bio_base% R:\model\TIMER_cc\user\%grid_user%\%project%\scenlib\%fscbase% /s /e /i

    # :COPYCCSCENTOR
    # echo.
    # echo Copy FSCbase scenario files to R location: %fscbase%
    # echo Copy FSCbase scenario files to R location: %% >> %log% 2>&1
    # sleep 3
    # xcopy /y %timer%\scenlib\%TIMERGeneration%\%project%\%fscbase% R:\model\TIMER_cc\user\%grid_user%\%project%\scenlib\%fscbase% /s /e /i

    # :SETTINGS1
    # sleep 3
    # echo off
    # set /a NR=1
    # echo set scenid = 1  > %timerccdir%\scenlib\%fscbase%\settings.scn
    # echo.
    # echo Scanning trunk settings: %trunk%
    # echo Scanning trunk settings: %trunk% >> %log% 2>&1
    # cd /d %timer%\scenlib\%TIMERGeneration%\baselines\%trunk%
    # set scen_name=%trunk%
    # call scenario_settings.bat
    # goto STARTSETTINGSFILE

    # :SETTINGS2
    # sleep 3
    # echo.
    # echo Scanning second baseline settings: %base_scen%
    # echo Scanning second baseline settings: %base_scen% >> %log% 2>&1
    # cd /d %timer%\scenlib\%TIMERGeneration%\baselines\%base_scen%
    # set scen_name=%base_scen%
    # call scenario_settings.bat
    # goto STARTSETTINGSFILE

    # :SETTINGS3
    # sleep 3
    # echo.
    # echo Scanning Bio base settings: %bio_base%
    # echo Scanning Bio base settings: %bio_base% >> %log% 2>&1
    # cd /d %timer%\scenlib\%TIMERGeneration%\baselines\%bio_base%
    # if not exist scenario_settings.bat goto SETTINGS4
    # set scen_name=%bio_base%
    # call scenario_settings.bat
    # goto STARTSETTINGSFILE

    # :SETTINGS4
    # sleep 3
    # echo.
    # echo Scanning FSCbase scenario settings: %fscbase%
    # echo Scanning FSCbase scenario settings: %fscbase% >> %log% 2>&1
    # cd /d %timer%\scenlib\%TIMERGeneration%\%project%\%fscbase%
    # if not exist scenario_settings.bat goto SETTINGS5
    # set scen_name=%fscbase%
    # call scenario_settings.bat
    # goto STARTSETTINGSFILE

    # :SETTINGS5
    # sleep 3
    # set /a NR=1
    # echo set DIRECTORY("../scenlib/$1/pop");  > %timerccdir%\scenlib\%fscbase%\scenario_input.sce
    # echo.
    # echo Scanning trunk sce-file: %trunk%
    # echo Scanning trunk sce-file: %trunk% >> %log% 2>&1
    # cd /d %timer%\scenlib\%TIMERGeneration%\baselines\%trunk%
    # goto STARTSCEFILE

    # :SCE2
    # sleep 3
    # echo.
    # echo Scanning second baseline sce-file: %base_scen%
    # echo Scanning second baseline sce-file: %base_scen% >> %log% 2>&1
    # cd /d %timer%\scenlib\%TIMERGeneration%\baselines\%base_scen%
    # goto STARTSCEFILE

    # :SCE3
    # sleep 3
    # echo.
    # echo Scanning Bio base sce-file: %bio_base%
    # echo Scanning Bio base sce-file: %bio_base% >> %log% 2>&1
    # cd /d %timer%\scenlib\%TIMERGeneration%\baselines\%bio_base%
    # goto STARTSCEFILE

    # :SCE4
    # sleep 3
    # echo.
    # echo Scanning additional scenario sce-file: %fscbase%
    # echo Scanning additional scenario sce-file: %fscbase% >> %log% 2>&1
    # cd /d %timer%\scenlib\%TIMERGeneration%\%project%\%fscbase%
    # goto STARTSCEFILE

    # :SCE5
    # echo off
    # sleep 3
    # echo.
    # echo Prepping batch script to start your run on grid
    # echo Prepping batch script to start your run on grid  >> %log% 2>&1
    # sleep 3

    # cd /d %timerccdir%
    # set STARTSCRIPT=Start_%fscbase%.bat

    # echo @echo on							 > %STARTSCRIPT%
    # echo=								>> %STARTSCRIPT%
    # echo set BATCH_DIR=R:\model\TIMER_cc\runBatches\TIMERhead 	>> %STARTSCRIPT%
    # echo set PREFIX_NAME=%fscbase% 				>> %STARTSCRIPT%
    # echo set SPEC_NAME=dummy 					>> %STARTSCRIPT%
    # echo set BASE_NAME=%fscbase%				>> %STARTSCRIPT%
    # echo set MODELZIP=%TIMERVersion%_r%svnrev%.zip 			>> %STARTSCRIPT%
    # echo set DIST_TYPE=4 						>> %STARTSCRIPT%
    # echo set PROJECTNAME=%project% 					>> %STARTSCRIPT%
    # echo=								>> %STARTSCRIPT%
    # echo :: ramp 							>> %STARTSCRIPT%
    # echo set CC_TYPE=ramp 						>> %STARTSCRIPT%
    # echo call %BATCH_DIR%\generatecc 				>> %STARTSCRIPT%
    # echo=								>> %STARTSCRIPT%
    # echo :: cubic 							>> %STARTSCRIPT%
    # echo set CC_TYPE=cubic 						>> %STARTSCRIPT%
    # echo call %BATCH_DIR%\generatecc 				>> %STARTSCRIPT%
    # echo=								>> %STARTSCRIPT%
    # echo :: cubicroot 						>> %STARTSCRIPT%
    # echo set CC_TYPE=cubicroot 					>> %STARTSCRIPT%
    # echo call %BATCH_DIR%\generatecc 				>> %STARTSCRIPT%

    # :READYTORUN
    # echo.
    # echo Ready to make some cost-curves
    # echo Ready to make some cost-curves >> %log% 2>&1
    # sleep 3
    # echo.
    # echo Good job, kostencurvebakker!
    # echo Good job, kostencurvebakker! >> %log% 2>&1
    # sleep 3

    # :END
    # cd /d %home%
    # exit /b 0

    # :COSTCURVESREADY
    # echo.
    # echo CostCurve are ready >> %log% 2>&1
    # echo CostCurve are ready 
    # sleep 3
    # echo.
    # echo Now producing >> %log% 2>&1
    # echo Now producing 
    # sleep 3
    # cd /d %timer%\Costcurve4K
    # call create_all_costcurves %fscbase% %project%

    # sleep 3
    # echo.
    # echo CostCurves ready >> %log% 2>&1
    # echo CostCurves ready 

    # :END
    # cd /d %home%
    # exit /b 0


    # :STARTSETTINGSFILE
    # for /F "delims=#" %%i in (in_%scen_name%) do call :MAKESETTINGSFILE "%%i"
    # del in_%scen_name%
    # set /a NR=%NR%+1
    # goto SETTINGS%NR%

    # :MAKESETTINGSFILE 
    # for /F "delims=#" %%i in (%1) do set line=%%i
    # echo %line% >> %timerccdir%\scenlib\%fscbase%\settings.scn
    # exit /b



    # :STARTSCEFILE
    # for /F "delims=#" %%i in (scenario_input.sce) do call :MAKESCEFILE "%%i"
    # set /a NR=%NR%+1
    # goto SCE%NR%

    # :MAKESCEFILE 
    # for /F "delims=#" %%i in (%1) do set line=%%i
    # echo %line% >> %timerccdir%\scenlib\%fscbase%\scenario_input.sce
    # exit /b


    # TODO: mapping:
    # construction: %trunk% [SSP2] -> %base_scen% [SSP1] -> %bio_base% [SSP1_REDD10reforFullP] -> %fscbase% [SSP1_450]
    # %trunk% -> baseline_kernel 
    # %base_scen% -> baseline_root (or should we rename to scenario_root?)
    # %bio_base% -> baseline_landuse
    # %fscbase% -> baseline_type    


# if __name__ == "__main__":
    # # timer_folder = os.path.join("Y:",os.sep,"Kennisbasis","IMAGE","model","users","maarten","011_Models","07 TIMER","TIMER_3v11","2_TIMER","TIMER_3_11_bunkers")
    # # import os
    # timer_branch = "TIMER_2015_SSPs"
    # timer_version = "TIMER_2015"
    # project = "SSPs"
    # scenario = "NPi10"
    # timer_folder = os.path.join(os.getcwd(),"..","..", timer_branch)
    # get_revision(timer_folder)
    # prepare_costcurve(timer_branch, timer_version, project, scenario)